In [ ]:
suppressMessages({
library(ArchR)
library(Signac)
library(Seurat)
library(GenomicRanges)
library(future)
library("EnsDb.Hsapiens.v86")
library(Signac)
library(Seurat)
library(ggplot2)
library(patchwork)
library(dplyr)
library(igraph)
library(tidyverse)
library(patchwork)
library(viridis)
library(scCustomize)
library(qs)
library(ggpubr)
library(clustree)
library("ggraph")
library("Nebulosa")
library(ggalluvial)
library("MAST")
library("pheatmap")
library("chromVAR")
library("TFBSTools")
#library("JASPAR2022")
library("motifmatchr")
library("presto")
library("JASPAR2022")
library(BSgenome.Hsapiens.UCSC.hg38)
library(ChIPseeker)
library(TxDb.Hsapiens.UCSC.hg38.knownGene)
library(Seurat)
library(sctransform)
library(dplyr)
#library(scDblFinder)
library(biomaRt) 
#library(scater)
library(SingleCellExperiment)
#library(SeuratDisk)
library(Signac)
library(GenomicRanges)
library(rtracklayer)
library(EnsDb.Hsapiens.v86)
library(scales)
library(future) 
#plan("multicore", workers = 10)
#options(future.globals.maxSize = 1000000 * 1024^2)
library(harmony)
library(biovizBase)
    })

In [ ]:
if (!dir.exists("Images")) {dir.create("Images")}
if (!dir.exists("Tables"))  {dir.create("Tables")}
if (!dir.exists("RObjects")) {dir.create("RObjects")}

IMG_OUT = "Images"
DF_OUT  = "Tables"
RDS_OUT = "RObjects"

In [ ]:
#basic steps to read in data and performing QC
    ##Creating a common peak set
    ### read in peak sets
    ### convert to genomic ranges
    ### Create a unified set of peaks to quantify in each dataset
    ### Filter out bad peaks based on length

    ##Create Fragment objects
    ### load metadata
    ### perform an initial filtering of low count cells
    ### create fragment objects

    ##Quantify peaks in each dataset
    
    ##Create the objects

    ##Annotate using EnsDb.Hsapiens.v86

    ##Perform basic QC based on nCount_ATAC, TSS.Enrichment and Nucleosomal.Signal
    
    ##Integrate datasets to be analyzed together
    

In [ ]:
# add information to identify dataset of origin
tscm.l1$Origin <- 'tscm.l1'
tscm.l2$Origin <- 'tscm.l2'

In [ ]:
# compute LSI
tscm.l1 <- FindTopFeatures(tscm.l1, min.cutoff = 10)
tscm.l1 <- RunTFIDF(tscm.l1)
tscm.l1 <- RunSVD(tscm.l1)

# compute LSI
tscm.l2 <- FindTopFeatures(tscm.l2, min.cutoff = 10)
tscm.l2 <- RunTFIDF(tscm.l2)
tscm.l2 <- RunSVD(tscm.l2)

In [ ]:
# find integration anchors
integration.anchors <- FindIntegrationAnchors(
  object.list = list(tscm.l1, tscm.l2),
  anchor.features = rownames(tscm.l2),
  reduction = "rlsi",
  dims = 2:30
)

# integrate LSI embeddings
integrated <- IntegrateEmbeddings(
  anchorset = integration.anchors,
  reductions = combined[["lsi"]],
  new.reduction.name = "integrated_lsi",
  dims.to.integrate = 1:30
)

# create a new UMAP using the integrated embeddings
integrated <- RunUMAP(integrated, reduction = "integrated_lsi", dims = 2:30, verbose = FALSE)
p2 <- DimPlot(integrated, group.by = "Origin")
p2

In [ ]:
# get gene annotations for hg38
suppressWarnings({
annotation <- GetGRangesFromEnsDb(ensdb = EnsDb.Hsapiens.v86)
seqlevels(annotation) <- paste0('chr', seqlevels(annotation))
seqlevelsStyle(annotation) <- 'UCSC'

Annotation(object = integrated) <- annotation
    })

In [ ]:
# call peaks using MACS2
peaks <- CallPeaks(seu.integrated)

# remove peaks on nonstandard chromosomes and in genomic blacklist regions
peaks <- keepStandardChromosomes(peaks, pruning.mode = "coarse")
peaks <- subsetByOverlaps(x = peaks, ranges = blacklist_hg38_unified, invert = TRUE)

# quantify counts in each peak
macs2_counts <- FeatureMatrix(
  fragments = Fragments(seu.integrated),
  features = peaks,
  cells = colnames(seu.integrated)
)

# create a new assay using the MACS2 peak set and add it to the Seurat object
seu.integrated[["peaks"]] <- CreateChromatinAssay(
  counts = macs2_counts,
  annotation = annotation,
  fragments = Fragments(seu.integrated)
)

In [ ]:
#use MACS2-called "peaks" assay to perform downstream analysis
DefaultAssay(seu.integrated) <- "peaks"
seu.integrated <- RunTFIDF(seu.integrated)
seu.integrated <- FindTopFeatures(seu.integrated, min.cutoff = 10)
seu.integrated <- RunSVD(seu.integrated)
seu.integrated <- RunUMAP(seu.integrated, dims = 2:30, reduction = 'integrated_lsi', metric = "euclidean", verbose = FALSE)

seu.integrated <- FindNeighbors(object = seu.integrated, reduction = 'integrated_lsi', dims = 2:30, graph.name = c("peaks_nn", "peaks_snn"))

seu.integrated <- FindClusters(object = seu.integrated, verbose = FALSE, algorithm = 3, resolution = 0.4, graph.name = "peaks_snn") 

In [ ]:
#generate umaps (Figure 3E)
Idents(seu.integrated) <- "peaks_snn_res.0.4"
pdf(sprintf("%s/umap.integrated.pdf", IMG_OUT), width=6, height=5)
DimPlot(seu.integrated, label = TRUE, repel = TRUE)
dev.off()

In [ ]:
#Create a gene activity matrix

gene.activities <- GeneActivity(seu.integrated)

# add the gene activity matrix to the Seurat object as a new assay and normalize it
seu.integrated[['RNA']] <- CreateAssayObject(counts = gene.activities)
seu.integrated <- NormalizeData(
  object = seu.integrated,
  assay = 'RNA',
  normalization.method = 'LogNormalize',
  scale.factor = median(seu.integrated$nCount_RNA)
)

In [ ]:
#prediction of scRNA-based clusters

In [ ]:
transfer.anchors <- FindTransferAnchors(reference = seu.rna, query = seu.integrated, features = VariableFeatures(object = seu.rna), 
    reference.assay = "RNA", query.assay = "RNA", reduction = "cca")

celltype.predictions <- TransferData(anchorset = transfer.anchors, refdata = seu.rna$SCT_snn_res.0.4, 
    weight.reduction = seu.integrated[["integrated_lsi"]], dims = 2:30)

seu.integrated <- AddMetaData(seu.integrated, metadata = celltype.predictions)

In [ ]:
#paint clustering information of scRNA data on predicted scATAC data (Figure 3G)
Idents(seu.integrated) <- "predicted.id"
seu.integrated@meta.data %>% filter(predicted.id %in% c("11")) %>% rownames() -> cl11.bcs
seu.integrated@meta.data %>% filter(predicted.id %in% c("12")) %>% rownames() -> cl12.bcs

#pdf(sprintf("%s/umap.predicted.id.cl11.12.highlight.pdf", IMG_OUT), width=7, height=5)
DimPlot(seu.integrated, cells.highlight = list(cl11.bcs, cl12.bcs), cols.highlight = c("#0f9ed5", "#78206e"), pt.size = 2, sizes.highlight = 2)
#dev.off()

In [ ]:
#generate cicero-based links

In [ ]:
seu.integrated[["UMAP"]] <- seu.integrated[["umap"]]
int.cds <- as.cell_data_set(x = seu.integrated)
umap_coords <- reducedDims(int.cds)$UMAP
int.cicero <- make_cicero_cds(int.cds, reduced_coordinates = umap_coords)

# get the chromosome sizes from the Seurat object
genome <- seqlengths(seu.integrated@assays$peaks@annotation)

# convert chromosome sizes to a dataframe
genome.df <- data.frame("chr" = names(genome), "length" = genome)

# run cicero
conns <- run_cicero(int.cicero, genomic_coords = genome.df, sample_num = 100)

ccans <- generate_ccans(conns)

links <- ConnectionsToLinks(conns = conns, ccans = ccans)
Links(seu.integrated) <- links

Links(seu.integrated) %>% as.data.frame() %>% write.csv("cicero.links.csv")

In [ ]:
#generate coverage plots (Figures 3H, 6G-H, S3D and S6A-B)
##coverage across clusters for genomic loci of interest
##violin plots (right) for gene activity for gene of interest
##peak plot corresponding to scATAC data
##Gene annotation for loci visualized
##ChIP-seq data as peak plot or BigWig track from publicly available dataset
##cicero-generated links
##H3K27ac from ENCODE
##H3K4me1 from ENCODE
##BigWig tracks from bulk-ATAC-Seq (10 donors) (only Figures 6G-6H and S6A-B)
##Peaks containing motif for TFs on interest (only Figure 6H)

In [ ]:
#gzmb (Figure 3H)
roi = "chr14-24600000-24645000"
cov_plot <- CoveragePlot(
  object = seu.integrated,
  region = roi,
  annotation = FALSE,
  peaks = FALSE,
  links = FALSE
)
#cov_plot

gene_plot <- AnnotationPlot(
  object = seu.integrated,
  region = roi
)
#gene_plot

peak_plot1 <- PeakPlot(
  object = seu.integrated,
  region = roi
)
#peak_plot1

peak_plot2 <- PeakPlot(seu.integrated, 
         region = roi, 
         peaks = GRanges(import("../eomes_chip_hg38.bed")), 
         color = "#DC268C")
#peak_plot2

link_plot <- LinkPlot(
  object = seu.integrated,
  region = roi
)
#link_plot

expr_plot <- ExpressionPlot(
  object = seu.integrated,
  features = "GZMB",
  assay = "RNA"
)
#expr_plot


chip2 <- BigwigTrack(region = roi, 
                     "../tbet_chip3_hg38.bigWig", 
                     ymax = "q95",
                     y_label = "T-bet") + scale_fill_manual(values="#921B63") + theme(legend.position="none")
chip3 <- BigwigTrack(region = roi, 
                     "../fosl2_r1.bigWig", 
                     ymax = "q95",
                     y_label = "FOSL2") + scale_fill_manual(values="#FA3939") + theme(legend.position="none")


chip5 <- BigwigTrack(roi, 
                     "../ENCODE/H3K27ac_CD4abT/1/ENCFF884NBE.bigWig",
                     y_label = "H3K27ac",
                     smooth = 100) + theme(legend.position="none") + scale_fill_manual(values="black")

chip6 <- BigwigTrack(roi, 
                     "../ENCODE/H3K4me1_CD45RO_CD4/ENCFF334TZP.bigWig",
                     y_label = "H3K4me1",
                     smooth = 100) + theme(legend.position="none") + scale_fill_manual(values="darkgrey")

CombineTracks(
  plotlist = list(cov_plot, peak_plot1, gene_plot, peak_plot2, chip2, chip3, link_plot, chip5, chip6),
  expression.plot = expr_plot,
  heights = c(10, 1, 2, 1, 2, 2, 3, 2, 2),
  widths = c(10, 1)
) -> gzmb.cov
gzmb.cov

pdf(sprintf("%s/gzmb.cov.pdf", IMG_OUT), width=8, height=8)
gzmb.cov
dev.off()

In [ ]:
#slamf7 (Figures 6G and S6A)
cov_plot <- CoveragePlot(
  object = seu.filt,
  region = roi4,
  annotation = FALSE,
  peaks = FALSE,
  links = FALSE
) & scale_fill_manual(values = c("#78206e", "#0f9ed5", "#FD95F8", "#80BB17", "#6C244C"))
#cov_plot

gene_plot <- AnnotationPlot(
  object = seu.filt,
  region = roi4
)
#gene_plot

peak_plot1 <- PeakPlot(
  object = seu.filt,
  region = roi4
)
#peak_plot1

peak_plot2 <- PeakPlot(seu.filt, 
         region = roi4, 
         peaks = GRanges(import("../eomes_chip_hg38.bed")), 
         color = "#DC268C")
#peak_plot2

link_plot <- LinkPlot(
  object = seu.filt,
  region = roi4,
  min.cutoff = 0.05
)
#link_plot

expr_plot <- ExpressionPlot(
  object = seu.filt,
  features = "SLAMF7",
  assay = "RNA"
) & scale_fill_manual(values = c("#78206e", "#0f9ed5", "#FD95F8", "#80BB17", "#6C244C"))
#expr_plot

peak_plot3 <- PeakPlot(seu.filt, 
         region = roi4, 
         peaks = bulk.gr, 
         color = "black")

gh <- PeakPlot(seu.filt, 
         region = roi4, 
         peaks = genehancer,
         group.by = "type", 
         color = c("#DE6012", "#5087A0"))

chip2 <- BigwigTrack(region = roi4, 
                     list("Tbet" = "../tbet_chip3_hg38.bigWig"), 
                     ymax = "q95") + scale_fill_manual(values="#921B63") + theme(legend.position="none")
chip3 <- BigwigTrack(region = roi4, 
                     list("FOSL2" = "../fosl2_r1.bigWig"), 
                     ymax = "q95") + scale_fill_manual(values="#FA3939") + theme(legend.position="none")

bulk <- BigwigTrack(region = roi4, 
                     list(
               "naive" = "../bulk_bw/N_out.bw",
               "temra_e" = "../bulk_bw/E_out.bw"), 
                     ymax = 5000) + scale_fill_manual(values=c("#7F7F7F", "#F8766D")) + theme(legend.position="none")

chip5 <- BigwigTrack(roi4, 
                     "../ENCODE/H3K27ac_CD4abT/1/ENCFF884NBE.bigWig",
                     y_label = "H3K27ac",
                     smooth = 100) + theme(legend.position="none") + scale_fill_manual(values="black")

chip6 <- BigwigTrack(roi4, 
                     "../ENCODE/H3K4me1_CD45RO_CD4/ENCFF334TZP.bigWig",
                     y_label = "H3K4me1",
                     smooth = 100) + theme(legend.position="none") + scale_fill_manual(values="darkgrey")
#chip5

CombineTracks(
  plotlist = list(cov_plot, peak_plot1, link_plot, bulk,
                  peak_plot3, gene_plot, peak_plot2, chip2, chip3, chip5, chip6),
  expression.plot = expr_plot,
  heights = c(15, 1, 3, 6, 1, 2, 1, 2, 2, 2, 2),
  widths = c(10, 1)
) -> slamf7.cov
slamf7.cov


In [ ]:
#extract motif containing peaks for TF of interest
pfm2 <- readJASPARMatrix("../CellRangerReferences/refdata-cellranger-arc-GRCh38-2020-A-2.0.0/regions/motifs.pfm", matrixClass="PFM")

In [ ]:
pfm.stat4 <- pfm2[grepl("STAT4", names(pfm2), ignore.case = TRUE)]
pfm.nfat5 <- pfm2[grepl("NFAT5", names(pfm2), ignore.case = TRUE)]
pfm.rel <- pfm2[grepl("REL_", names(pfm2), ignore.case = TRUE)]
pfm.arntl <- pfm2[grepl("ARNTL", names(pfm2), ignore.case = TRUE)]
pfm.klf12 <- pfm2[grepl("KLF12", names(pfm2), ignore.case = TRUE)]
pfm.tcf12 <- pfm2[grepl("TCF12_", names(pfm2), ignore.case = TRUE)]
pfm.foxn3 <- pfm2[grepl("FOXN3", names(pfm2), ignore.case = TRUE)]
pfm.nfkb1 <- pfm2[grepl("NFKB1", names(pfm2), ignore.case = TRUE)]
pfm.cux1 <- pfm2[grepl("CUX1", names(pfm2), ignore.case = TRUE)]
pfm.runx2 <- pfm2[grepl("RUNX2", names(pfm2), ignore.case = TRUE)]
pfm.nr3c1 <- pfm2[grepl("NR3C1", names(pfm2), ignore.case = TRUE)]
pfm.smad3 <- pfm2[grepl("SMAD3", names(pfm2), ignore.case = TRUE)]
pfm.nfatc3 <- pfm2[grepl("NFATC3", names(pfm2), ignore.case = TRUE)]

In [ ]:
motif_pos_stat4 <- matchMotifs(pfm.stat4, StringToGRanges(rownames(seu.filt[["peaks"]])), 
                                   genome = BSgenome.Hsapiens.UCSC.hg38,  out="positions", p.cutoff=0.0005)

motif_pos_nfat5 <- matchMotifs(pfm.nfat5, StringToGRanges(rownames(seu.filt[["peaks"]])), 
                                   genome = BSgenome.Hsapiens.UCSC.hg38,  out="positions", p.cutoff=0.0005)

motif_pos_rel <- matchMotifs(pfm.rel, StringToGRanges(rownames(seu.filt[["peaks"]])), 
                                   genome = BSgenome.Hsapiens.UCSC.hg38,  out="positions", p.cutoff=0.0005)

motif_pos_arntl <- matchMotifs(pfm.arntl, StringToGRanges(rownames(seu.filt[["peaks"]])), 
                                   genome = BSgenome.Hsapiens.UCSC.hg38,  out="positions", p.cutoff=0.0005)

motif_pos_klf12 <- matchMotifs(pfm.klf12, StringToGRanges(rownames(seu.filt[["peaks"]])), 
                                   genome = BSgenome.Hsapiens.UCSC.hg38,  out="positions", p.cutoff=0.0005)

motif_pos_tcf12 <- matchMotifs(pfm.tcf12, StringToGRanges(rownames(seu.filt[["peaks"]])), 
                                   genome = BSgenome.Hsapiens.UCSC.hg38,  out="positions", p.cutoff=0.0005)

motif_pos_foxn3 <- matchMotifs(pfm.foxn3, StringToGRanges(rownames(seu.filt[["peaks"]])), 
                                   genome = BSgenome.Hsapiens.UCSC.hg38,  out="positions", p.cutoff=0.0005)

motif_pos_nfkb1 <- matchMotifs(pfm.nfkb1, StringToGRanges(rownames(seu.filt[["peaks"]])), 
                                   genome = BSgenome.Hsapiens.UCSC.hg38,  out="positions", p.cutoff=0.0005)

motif_pos_cux1 <- matchMotifs(pfm.cux1, StringToGRanges(rownames(seu.filt[["peaks"]])), 
                                   genome = BSgenome.Hsapiens.UCSC.hg38,  out="positions", p.cutoff=0.0005)

motif_pos_runx2 <- matchMotifs(pfm.runx2, StringToGRanges(rownames(seu.filt[["peaks"]])), 
                                   genome = BSgenome.Hsapiens.UCSC.hg38,  out="positions", p.cutoff=0.0005)

motif_pos_nr3c1 <- matchMotifs(pfm.nr3c1, StringToGRanges(rownames(seu.filt[["peaks"]])), 
                                   genome = BSgenome.Hsapiens.UCSC.hg38,  out="positions", p.cutoff=0.0005)

motif_pos_smad3 <- matchMotifs(pfm.smad3, StringToGRanges(rownames(seu.filt[["peaks"]])), 
                                   genome = BSgenome.Hsapiens.UCSC.hg38,  out="positions", p.cutoff=0.0005)

motif_pos_nfatc3 <- matchMotifs(pfm.nfatc3, StringToGRanges(rownames(seu.filt[["peaks"]])), 
                                   genome = BSgenome.Hsapiens.UCSC.hg38,  out="positions", p.cutoff=0.0005)

In [ ]:
motif_pos_stat4$Stat4_MA0518.1 %>% filter(score>10) -> motif_pos_stat4_filt1
motif_pos_nfat5$NFAT5_MA0606.1 %>% filter(score>10) -> motif_pos_nfat5_filt1
motif_pos_rel$REL_MA0101.1 %>% filter(score>10) -> motif_pos_rel_filt1
motif_pos_arntl$Arntl_MA0603.1 %>% filter(score>10) -> motif_pos_arntl_filt1
motif_pos_klf12$Klf12_MA0742.1 %>% filter(score>10) -> motif_pos_klf12_filt1
motif_pos_tcf12$Tcf12_MA0521.1 %>% filter(score>10) -> motif_pos_tcf12_filt1
motif_pos_foxn3$FOXN3_MA1489.1 %>% filter(score>10) -> motif_pos_foxn3_filt1
motif_pos_nfkb1$NFKB1_MA0105.4 %>% filter(score>10) -> motif_pos_nfkb1_filt1
motif_pos_cux1$CUX1_MA0754.1 %>% filter(score>10) -> motif_pos_cux1_filt1
motif_pos_runx2$RUNX2_MA0511.2 %>% filter(score>10) -> motif_pos_runx2_filt1
motif_pos_nr3c1$NR3C1_MA0113.3 %>% filter(score>10) -> motif_pos_nr3c1_filt1
motif_pos_smad3$SMAD3_MA0795.1 %>% filter(score>10) -> motif_pos_smad3_filt1
motif_pos_nfatc3$NFATC3_MA0625.1 %>% filter(score>10) -> motif_pos_nfatc3_filt1


In [ ]:
motif_pos_stat4_filt1$tf <- "Stat4"
motif_pos_nfat5_filt1$tf <- "NFAT5"
motif_pos_rel_filt1$tf <- "REL"
motif_pos_arntl_filt1$tf <- "Arntl"
motif_pos_klf12_filt1$tf <- "Klf12"
motif_pos_tcf12_filt1$tf <- "Tcf12"
motif_pos_foxn3_filt1$tf <- "FOXN3"
motif_pos_nfkb1_filt1$tf <- "NFKB1"
motif_pos_cux1_filt1$tf <- "CUX1"
motif_pos_runx2_filt1$tf <- "RUNX2"
motif_pos_nr3c1_filt1$tf <- "NR3C1"
motif_pos_smad3_filt1$tf <- "SMAD3"
motif_pos_nfatc3_filt1$tf <- "NFATC3"

c(motif_pos_stat4_filt1, motif_pos_nfat5_filt1, motif_pos_rel_filt1, motif_pos_arntl_filt1,
  motif_pos_klf12_filt1, motif_pos_tcf12_filt1, motif_pos_foxn3_filt1, motif_pos_nfkb1_filt1,
  motif_pos_cux1_filt1, motif_pos_runx2_filt1, motif_pos_nr3c1_filt1, motif_pos_smad3_filt1,
  motif_pos_nfatc3_filt1) -> ranges.ofint

In [ ]:
require(purrr)
require(stringr)
require(dplyr)
require(ggplot2)

# this randomly generates a color
rand_color <- function(){
  1:3 %>% 
    map_chr(~ str_pad(as.hexmode(floor(runif(1,0,256))),2,pad="0")) %>%
    paste0(collapse="") %>% 
    paste0("#",.)
}

# this is a function that, given a number, returns that number of colors
my_colors <- function(n) 1:n %>% map_chr(~ rand_color())

In [ ]:
limits <- c("Stat4","NFAT5","REL","Arntl","Klf12","Tcf12","FOXN3","NFKB1","CUX1","RUNX2","NR3C1","SMAD3","NFATC3")
tfcols <- c('#f03111','#b3d1f8','#4d27a9','#0d52a1','#c21976','#464670','#b22a1e','#764907','#7dd615','#ea26d6','#4c9dfa','#9e538e','#0814f7')

In [ ]:
#fosl2 (Figures 6H and S6B)
cov_plot <- CoveragePlot(
  object = seu.filt,
  region = roi8,
  annotation = FALSE,
  peaks = FALSE,
  links = FALSE
) & scale_fill_manual(values = c("#78206e", "#0f9ed5", "#FD95F8", "#80BB17", "#6C244C"))
#cov_plot

gene_plot <- AnnotationPlot(
  object = seu.filt,
  region = roi8
)
#gene_plot

peak_plot1 <- PeakPlot(
  object = seu.filt,
  region = roi8
)
#peak_plot1

peak_plot2 <- PeakPlot(seu.filt, 
         region = roi8, 
         peaks = ranges.ofint, 
         group.by = "tf") & scale_color_manual(limits = limits, values = tfcols) & theme(legend.position="none")
#peak_plot2

link_plot <- LinkPlot(
  object = seu.filt,
  region = roi8,
  min.cutoff = 0.05
)
#link_plot

expr_plot <- ExpressionPlot(
  object = seu.filt,
  features = "FOSL2",
  assay = "RNA"
) & scale_fill_manual(values = c("#78206e", "#0f9ed5", "#FD95F8", "#80BB17", "#6C244C"))
#expr_plot

peak_plot3 <- PeakPlot(seu.filt, 
         region = roi8, 
         peaks = bulk.gr, 
         color = "black")

bulk <- BigwigTrack(region = roi8, 
                     list(
               "naive" = "../bulk_bw/N_out.bw",
               "temra_e" = "../bulk_bw/E_out.bw"), 
                     ymax = 5000) + scale_fill_manual(values=c("#7F7F7F", "#F8766D")) + theme(legend.position="none")

chip5 <- BigwigTrack(roi8, 
                     "../ENCODE/H3K27ac_CD4abT/1/ENCFF884NBE.bigWig",
                     y_label = "H3K27ac",
                     smooth = 100) + theme(legend.position="none") + scale_fill_manual(values="black")

chip6 <- BigwigTrack(roi8, 
                     "../ENCODE/H3K4me1_CD45RO_CD4/ENCFF334TZP.bigWig",
                     y_label = "H3K4me1",
                     smooth = 100) + theme(legend.position="none") + scale_fill_manual(values="darkgrey")
#chip5

CombineTracks(
  plotlist = list(cov_plot, peak_plot1, link_plot, bulk,
                  peak_plot3, gene_plot, peak_plot2, chip5, chip6),
  expression.plot = expr_plot,
  heights = c(15, 1, 3, 6, 1, 2, 2, 2, 2),
  widths = c(10, 1)
) -> fosl2.cov
fosl2.cov


In [ ]:
#module score analysis (Figures 3F, 6F and S3C)

In [ ]:
seu.integrated -> abc

In [ ]:
bulk.lists <- parse_marker_gmt("GMT/bulk.lists.narrowed.gmt", trim_first=FALSE)

In [ ]:
DefaultAssay(abc) <- "peaks"
RegionStats(
  object = abc,
  genome = BSgenome.Hsapiens.UCSC.hg38,
  assay = "peaks"
) -> def

suppressWarnings({
LinkPeaks(def,
         peak.assay = "peaks",
         expression.assay = "RNA"
         ) -> def
    })

In [ ]:
GetLinkedPeaks(def, features = bulk.lists[["EvN.top200"]], assay = "peaks") -> evn.peaks
features <- list("EvN.top200" = evn.peaks)

# pathway scores
def -> ghi
	ghi <- AddChromatinModule (
		ghi, features = features, genome = BSgenome.Hsapiens.UCSC.hg38
	)

In [ ]:
vlistViolin1 <- list()

for (kp in vlist.enrich.colnames)
{
    plot_violin_metadata(ghi@meta.data[, c(29, 48:53)], "peaks_snn_res.0.4", kp) -> vlistViolin1[[kp]]
}

#violin plots for columns from metadata

print_plots_in_list(vlistViolin1, title = "VLIST1")

In [ ]:
#generate heatmap of activity for groups of cells (Figure 6F)

In [ ]:
HMAP4.act.de <- function(obj, filtered_de, subset_barcodes=NULL, cell.groups="seurat_clusters")
{
    if (!is.null(subset_barcodes)) {
        obj <- obj[, subset_barcodes]
    }
    intersect(unique(filtered_de),rownames(obj@assays$RNA@data)) -> genes.heatmap
    c("no.cytotox", "low.cytotox", "high.cytotox") -> clusters.heatmap 
    heatmap.list <- list()
    for (j in clusters.heatmap) {
        select_cells_by <- sprintf("%s == %s", cell.groups, "j")
        obj@meta.data %>% filter(!! rlang::parse_expr(select_cells_by)) %>% rownames() -> cls.cells
        obj@assays$RNA@data[genes.heatmap, cls.cells] %>% rowMeans2(useNames = FALSE) -> heatmap.list[[j]]
    }
    matrix(
        unlist(heatmap.list), 
        ncol=length(clusters.heatmap), nrow=length(genes.heatmap)
    ) %>% t() %>% scale() %>% t() -> mat.hmap
    rownames(mat.hmap) <- genes.heatmap
    colnames(mat.hmap) <- clusters.heatmap
   # return(mat.hmap)
    #mat.hmap <- mat.hmap[, c("0", "1", "2", "3", "4", "5", "6", "7", "8", "9", "10", "11", "12", "13", "14", "15", "16")]
    col_fun_act = paletteContinuous(set = "blueYellow", n = 5, reverse = FALSE)
    #col_fun_act(seq(-3, 3))
    Heatmap(
        mat.hmap, 
        cluster_rows=T,
        cluster_columns=FALSE, 
        col= col_fun_act,
        column_names_rot = 0, column_names_centered = T, row_names_gp = gpar(fontsize = 4),
        heatmap_legend_param = list(legend_direction = "vertical")
    ) -> p.n
    #draw(p.n, heatmap_legend_side="bottom")
}

In [ ]:
#pdf(sprintf("%s/hmap.cyto.activity.pdf", IMG_OUT), width=6, height=20)
HMAP4.act.de(def,  filtered_de = geneset[["Cytotoxicity"]], cell.groups = "type")
#dev.off()